# Importar Modulos

In [42]:
import pandas as pd
import numpy as np
import seaborn as sn
import plotly.express as px
import tensorflow as tf

from sklearn.model_selection import train_test_split
from IPython.display import display
import keras_tuner as kt

# Importación de Datos

In [43]:
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.3f}".format

data = np.loadtxt("lp5.csv", delimiter=",", dtype=str)
data_copy = data #va de 0 a 2623 o sea 2624 datos
cont = 16
labels = []
numbers = []
features= np.zeros((164,15,6))

for i in range(len(data_copy)):
    if(cont == 16):
        labels.append(data_copy[i])
    if(cont<16):
        numbers.append(data_copy[i])
    cont -= 1
    if(cont == 0):
        cont = 16
        
cont = 0
for i in range(164):
    for j in range(15):
        for z in range(6):
            features[i][j][z] = numbers[cont][z]
        cont += 1
        
labels_df = pd.DataFrame(labels, columns=['labels','1','2','3','4','5'])
#labels_df = labels_df.loc[:,['labels']]
display(labels_df)
display(features)

,labels,1,2,3,4,5
0,normal,,,,,
1,normal,,,,,
2,normal,,,,,
3,normal,,,,,
4,normal,,,,,
...,...,...,...,...,...,...
159,collision_in_tool,,,,,
160,collision_in_tool,,,,,
161,collision_in_tool,,,,,
162,collision_in_tool,,,,,


array([[[ -2.,  -1.,  81.,   0.,  -5.,   0.],
        [ -2.,  -1.,  79.,   0.,  -4.,   0.],
        [ -2.,  -1.,  79.,   0.,  -4.,   0.],
        ...,
        [ -2.,  -1.,  78.,   0.,  -5.,   0.],
        [ -3.,  -1.,  80.,   1.,  -4.,   1.],
        [ -2.,  -1.,  79.,   0.,  -4.,   0.]],

       [[  6.,  -1.,  79.,  -2.,   4.,  -3.],
        [ 42.,  -3.,  80.,   5.,  53.,   3.],
        [ -5.,   4.,  74., -15., -10.,  -1.],
        ...,
        [ -1.,  -5.,  80.,   6.,  -6.,   0.],
        [ -4.,   5.,  78., -14.,  -9.,  -4.],
        [ -4.,   1.,  80.,  -3., -12.,   5.]],

       [[ -2.,  -6.,  85.,  14.,  -5.,   2.],
        [  0.,   2.,  74.,  -7.,   1.,   0.],
        [ -4.,  -5.,  76.,   7., -11.,   4.],
        ...,
        [  0.,  -9.,  87.,  13.,  -5.,   2.],
        [ -5.,   5.,  67., -17., -16.,   7.],
        [ -6., -10.,  86.,  16., -14.,  -1.]],

       ...,

       [[-12.,  17.,   3., -19., -10.,  -4.],
        [-12.,  12.,  11., -13., -16.,  -4.],
        [ -8.,   3.,  

# Manejo de Datos y Normalización

In [44]:
classes_dict = {'normal':'1 0 0 0 0', #1
        'collision_in_tool':'0 1 0 0 0', #2
        'collision_in_part':'0 0 1 0 0', #3
        'bottom_collision':'0 0 0 1 0', #4
        'bottom_obstruction':'0 0 0 0 1'} #5

labels_df = labels_df.replace({'labels':classes_dict})
labels_df[['1', '2', '3', '4', '5']] = labels_df['labels'].str.split(' ', 4, expand= True)
labels_df = labels_df.loc[:,['1', '2', '3', '4', '5']]
labels_df[['1', '2', '3', '4', '5']]=labels_df[['1', '2', '3', '4', '5']].astype(str).astype(int)

def norm(x):
    x_min = x.min()
    x_max = x.max()
    range = x_max - x_min  #min max entre 0 y 1
    return((x-x_min)/(range))

norm_features = norm(features)
#split para entrenamiento y validacion
train_features, test_features, train_labels, test_labels  = train_test_split(norm_features, labels_df, test_size=0.2, random_state= 42)
display(train_features)
display(train_labels)

C:\Users\dave4\AppData\Local\Temp\ipykernel_7200\1747463221.py:8: FutureWarning:

In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.



array([[[0.77660263, 0.78156702, 0.79084826, 0.77055903, 0.7684006 ,
         0.77422836],
        [0.77466005, 0.78739478, 0.78717893, 0.75156486, 0.7638679 ,
         0.77206993],
        [0.77401252, 0.78804231, 0.79041658, 0.74465789, 0.76106195,
         0.77012735],
        ...,
        [0.78005612, 0.7804878 , 0.78437298, 0.78135118, 0.78113533,
         0.78027196],
        [0.78027196, 0.78091949, 0.77962443, 0.78091949, 0.78027196,
         0.78027196],
        [0.78027196, 0.78070365, 0.78286208, 0.78156702, 0.78091949,
         0.78005612]],

       [[0.78027196, 0.77940859, 0.79905029, 0.78372545, 0.77962443,
         0.78113533],
        [0.78070365, 0.78113533, 0.79667602, 0.77919275, 0.78091949,
         0.78070365],
        [0.77984028, 0.77962443, 0.79710771, 0.78221455, 0.77832938,
         0.78156702],
        ...,
        [0.78070365, 0.77876106, 0.79948198, 0.78350961, 0.77962443,
         0.78113533],
        [0.77962443, 0.78178286, 0.79516512, 0.77703432, 0.777

,1,2,3,4,5
84,0,0,1,0,0
2,1,0,0,0,0
94,0,1,0,0,0
45,0,0,1,0,0
42,0,0,1,0,0
...,...,...,...,...,...
71,0,0,1,0,0
106,0,0,0,1,0
14,1,0,0,0,0
92,0,1,0,0,0


# Creación del Modelo

In [45]:
'''def my_model(hp):
    inputs = tf.keras.Input(shape=(15,6,1))
    x = inputs
    for i in range(hp.Int('conv_blocks', 2, 5, default=2)):
        filters = hp.Int('filters_' + str(i), 8, 256, step=32) 
        
    for i in range(2):
        x = tf.keras.layers.Conv2D(filters, kernel_size= hp.Choice('conv_1_kernel', values = [3,5]), padding= 'same')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.ReLU()(x)
        
    if hp.Choice('pooling_' + str(i), ['avg', 'max']) == 'max': # hp.Choice chooses from a list of values
        x = tf.keras.layers.MaxPool2D()(x)
    else:
        x = tf.keras.layers.AvgPool2D()(x)

    x = tf.keras.layers.GlobalAvgPool2D()(x) # apply GlobalAvG Pooling
    
    # Tune the number of units in the  Dense layer
    # Choose an optimal value between min_value to max_value
    x = tf.keras.layers.Dense(hp.Int('Dense units',10, 100, step=10, default=50), activation='relu')(x)
    outputs = tf.keras.layers.Dense(5, activation= 'softmax')(x) # output layer 

    # define the model
    model = tf.keras.Model(inputs, outputs)

    # Tune the learning rate for the optimizer
    # Choose an optimal value frommin_value to max_value
    model.compile(optimizer= tf.keras.optimizers.Adam(hp.Float('learning_rate',1e-4, 1e-2, sampling='log')), 
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics = ['accuracy'])
    return model

# initialize tuner to run the model.
# using the Hyperband search algorithm
tuner = kt.Hyperband(
    hypermodel = my_model,
    objective='val_accuracy',
    max_epochs=30,
    hyperband_iterations=2,
    directory="Keras_tuner_dir",
    project_name="Keras_tuner_Demo")

# Run the search
tuner.search(train_features, train_labels,
            validation_data= (test_features,test_labels), 
            epochs=30,
            callbacks=[tf.keras.callbacks.EarlyStopping(patience=2)])

# Get the optimal hyperparameters
best_hps= tuner.get_best_hyperparameters(1)[0]

# get the best model
best_model = tuner.get_best_models(1)[0]

# show model summary
best_model.summary()'''
def my_model(hp):
    filters1=hp.Int('conv_filt1', 4, 16, step=4)
    filter2=hp.Int('conv_filt2', 8, 32, step=8)
    kernel1 = hp.Choice('kernel1', values = [3,5])
    kernel2 = hp.Choice('kernel2', values = [3,5])
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(filters1, kernel_size=kernel1, activation='relu', input_shape = (15,6,1), padding='same'), # #de feature maps, dimensiones del kernel, kernel entre más pequeño mejor y se prefiere un número impar
        tf.keras.layers.Conv2D(filters1, kernel_size=kernel1, activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2,2), strides= 2, padding= 'valid'), # dimensiones del pooling
        tf.keras.layers.Dropout(hp.Float('dropout1',0.25, 1.0)),
        tf.keras.layers.Conv2D(filter2, kernel_size=kernel2, activation='relu', padding='same'),
        tf.keras.layers.Conv2D(filter2, kernel_size=kernel2, activation='relu', padding='same'),# #de feature maps, dimensiones del kernel, kernel entre más pequeño mejor y se prefiere un número impar
        tf.keras.layers.MaxPooling2D((2,2), strides= 2, padding= 'valid'), # dimensiones del pooling
        tf.keras.layers.Dropout(hp.Float('dropout2',0.15, 1.0)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=hp.Int('neuronas', 10, 30, step=5), activation='relu'),
        tf.keras.layers.Dense(5, activation='softmax')])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Float('learning_rate',0.0005, 0.005)), loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
    
    return model

tuner = kt.GridSearch(
    hypermodel = my_model,
    objective='val_accuracy',
    max_trials= 200,
    directory="Tuner_Conv",
    project_name="Tarea1_B",
    overwrite = True)

tuner.search(train_features, train_labels,
            validation_data= (test_features,test_labels), 
            epochs=50,
            callbacks=[tf.keras.callbacks.EarlyStopping(patience=2)])

# Get the optimal hyperparameters
best_hps= tuner.get_best_hyperparameters(1)[0]

# get the best model
best_model = tuner.get_best_models(1)[0]

# show model summary
best_model.summary()
print(best_hps)

model = tuner.hypermodel.build(best_hps)

Trial 200 Complete [00h 00m 02s]
val_accuracy: 0.3333333432674408

Best val_accuracy So Far: 0.3333333432674408
Total elapsed time: 00h 06m 52s
INFO:tensorflow:Oracle triggered exit


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 15, 6, 8)          80        
                                                                 
 conv2d_1 (Conv2D)           (None, 15, 6, 8)          584       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 7, 3, 8)          0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 7, 3, 8)           0         
                                                                 
 conv2d_2 (Conv2D)           (None, 7, 3, 16)          1168      
                                                                 
 conv2d_3 (Conv2D)           (None, 7, 3, 16)          2320      
                                                        

# Entrenamiento del Modelo

In [46]:
def train_model(model, features, labels, epochs, batch_size):
    history = model.fit(
        x = features,
        y = labels,
        epochs= epochs,
        batch_size= batch_size,
        validation_split= 0.25
    )
    hist= pd.DataFrame(history.history) #se guardan los valores de errores y metricas en un diccionario
    hist['epoch'] = history.epoch #los epochs se deben añadir aparte
    return hist

# Visualizaciones
### Pasos para visualizar los filtros:
##### 1. Iterar por todas las capas del modelo usando model.layers
##### 2. Si la capa actual es de convolucion se extraen los pesos y sesgos usando get_weights()
##### 3. Se normalizan los pesos de los filtros entre 0 y 1
##### 4. Se plotean los filtros para cada capa convolutional y todos los canales de color

In [47]:
def loss_curves(history):
    hist = history
    labels = {"loss":"Training Loss", "val_loss":"Validation Loss"}
    hist.rename(columns = labels, inplace = True)
    
    fig = px.line(hist, x='epoch', y=['Training Loss', 'Validation Loss'],
                title='Gráficas de Pérdida de Entrenamiento y Evaluación',
                labels={"epoch": "Epoch", "value":"Binary Cross Entropy", "variable":"Curvas de Pérdida"},
                color_discrete_map={ # replaces default color mapping by value
                "Training Loss": "#46039f", "Validation Loss": "#fb9f3a"})
    fig.update_layout(template='plotly_white')
    fig.show()

# Se corren las funciones

In [48]:
learning_rate = 0.01
epochs = 200
batch_size = 1
#model = my_model(learning_rate)
history= train_model(model, train_features, train_labels, epochs, batch_size)
display(history)
loss_curves(history)
#view_filters(model)
display(history)

Epoch 1/250


C:\Users\dave4\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\backend.py:5703: UserWarning:

"`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?



98/98 [==============================] - 1s 3ms/step - loss: 0.5530 - accuracy: 0.2653 - val_loss: 0.4961 - val_accuracy: 0.2727
Epoch 2/250
40/98 [===========>..................] - ETA: 0s - loss: 0.5238 - accuracy: 0.2000    

C:\Users\dave4\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\backend.py:5703: UserWarning:

"`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?



98/98 [==============================] - 0s 2ms/step - loss: 0.5043 - accuracy: 0.2857 - val_loss: 0.5014 - val_accuracy: 0.2727
Epoch 3/250
98/98 [==============================] - 0s 2ms/step - loss: 0.5126 - accuracy: 0.3061 - val_loss: 0.4863 - val_accuracy: 0.2727
Epoch 4/250
98/98 [==============================] - 0s 2ms/step - loss: 0.4940 - accuracy: 0.3061 - val_loss: 0.4951 - val_accuracy: 0.2727
Epoch 5/250
98/98 [==============================] - 0s 2ms/step - loss: 0.4980 - accuracy: 0.2755 - val_loss: 0.4953 - val_accuracy: 0.2727
Epoch 6/250
98/98 [==============================] - 0s 2ms/step - loss: 0.4942 - accuracy: 0.3163 - val_loss: 0.4957 - val_accuracy: 0.2727
Epoch 7/250
98/98 [==============================] - 0s 2ms/step - loss: 0.4932 - accuracy: 0.3163 - val_loss: 0.4926 - val_accuracy: 0.2727
Epoch 8/250
98/98 [==============================] - 0s 2ms/step - loss: 0.4930 - accuracy: 0.3061 - val_loss: 0.4877 - val_accuracy: 0.2727
Epoch 9/250
98/98 [======

,loss,accuracy,val_loss,val_accuracy,epoch
0,0.553,0.265,0.496,0.273,0
1,0.504,0.286,0.501,0.273,1
2,0.513,0.306,0.486,0.273,2
3,0.494,0.306,0.495,0.273,3
4,0.498,0.276,0.495,0.273,4
...,...,...,...,...,...
245,0.444,0.449,0.412,0.364,245
246,0.378,0.480,0.397,0.394,246
247,0.365,0.480,0.388,0.455,247
248,0.356,0.500,0.360,0.455,248


,Training Loss,accuracy,Validation Loss,val_accuracy,epoch
0,0.553,0.265,0.496,0.273,0
1,0.504,0.286,0.501,0.273,1
2,0.513,0.306,0.486,0.273,2
3,0.494,0.306,0.495,0.273,3
4,0.498,0.276,0.495,0.273,4
...,...,...,...,...,...
245,0.444,0.449,0.412,0.364,245
246,0.378,0.480,0.397,0.394,246
247,0.365,0.480,0.388,0.455,247
248,0.356,0.500,0.360,0.455,248


# Predicciones

In [49]:
predictions = model.predict(test_features)
predictions_df = pd.DataFrame(predictions, columns=['normal', 'collision_in_tool', 'collision_in_part', 'bottom_collision', 'bottom_obstruction'])
display(predictions_df)
display(test_labels)

2/2 [==============================] - 0s 2ms/step


,normal,collision_in_tool,collision_in_part,bottom_collision,bottom_obstruction
0,0.179,0.161,0.445,0.192,0.024
1,0.000,0.000,0.000,0.000,1.000
2,0.000,0.000,0.000,0.984,0.015
3,0.378,0.155,0.402,0.059,0.006
4,0.297,0.158,0.443,0.092,0.010
...,...,...,...,...,...
28,0.081,0.129,0.400,0.347,0.044
29,0.332,0.157,0.426,0.076,0.008
30,0.170,0.158,0.446,0.201,0.024
31,0.324,0.160,0.427,0.081,0.009


,1,2,3,4,5
135,0,0,0,1,0
115,0,0,0,0,1
131,0,0,0,0,1
55,1,0,0,0,0
95,0,1,0,0,0
...,...,...,...,...,...
134,0,0,0,1,0
160,0,1,0,0,0
139,0,0,0,1,0
78,0,0,1,0,0
